## OpenMMTools Integration

This enables OPS to use any testsystem from openmmtools to be used as well as add ANY possible OpenMM setup, not just the ones that we provided before. The old OpenMMEngine has been rename intp `SimpleOpenMMEngine`

In [1]:
from __future__ import print_function

import openpathsampling as paths

try:
    import openmm as omm
    import openmm.unit as u
except ImportError: # OpenMM < 7.6
    import simtk.openmm as omm
    import simtk.unit as u
import openmmtools as omt
import mdtraj as md

import openpathsampling.engines.openmm as eng


Create Alanine in Vacuum and run it using OPS.

In [2]:
testsystem = omt.testsystems.AlanineDipeptideVacuum()

Let's have a look at the content

In [3]:
# NBVAL_SKIP
{ key: type(value) for key, value in testsystem.__dict__.items()}

{'_system': openmm.openmm.System,
 '_positions': openmm.unit.quantity.Quantity,
 '_topology': openmm.app.topology.Topology,
 '_mdtraj_topology': NoneType}

An `OpenMM` simulation in OPS needs 3 ingredients to function
1. A template snapshot (which contains topolgy and coordinates/velocities), 
2. an OpenMM system object, and 
3. an OpenMM integrator object.

In the following we will get these parts.

#### 1. The template

The information for the template is contained in the `_topology` and the `_positions` object.

In [4]:
template = eng.snapshot_from_testsystem(testsystem)

#### 2. The system

This is easy since we directly get it from the testsytem objects

In [5]:
system = testsystem.system

#### 3. The integrator

The openmmtools system is (almost) independent of the integrator and `openmm` provides us with lots of options. For now we pick something simple.

In [6]:
integrator = omm.VerletIntegrator(
    0.002 * u.picoseconds    
)

build the engine

In [7]:
engine = eng.Engine(
    template.topology, 
    system, 
    integrator)

Let's run a simulation of 10 steps.

In [8]:
traj = engine.generate(template, [paths.LengthEnsemble(10).can_append])

In [9]:
assert len(traj) == 10

In [10]:
# NBVAL_SKIP
print(traj[5].coordinates)

[[ 0.18766736  0.11599679  0.01245004]
 [ 0.19458341  0.22444927  0.00401414]
 [ 0.1443672   0.27122689  0.08869714]
 [ 0.14886747  0.25747473 -0.08926164]
 [ 0.33994537  0.26638274  0.00609843]
 [ 0.42709578  0.1897011   0.04380051]
 [ 0.36886493  0.38998331 -0.03181562]
 [ 0.293367    0.452431   -0.05633815]
 [ 0.49859312  0.4565761  -0.02400124]
 [ 0.5581438   0.41515287  0.0573552 ]
 [ 0.5732535   0.43391328 -0.15561179]
 [ 0.5191127   0.48312774 -0.23640657]
 [ 0.67427683  0.47320502 -0.14414124]
 [ 0.58423056  0.32728151 -0.17536463]
 [ 0.4749522   0.60669039 -0.00127668]
 [ 0.35958399  0.64924202  0.00474684]
 [ 0.5835927   0.68379167  0.00902624]
 [ 0.6747424   0.64066121  0.00332412]
 [ 0.57127444  0.82897221  0.01853087]
 [ 0.46718868  0.86095438  0.02347777]
 [ 0.62214258  0.86301533  0.10872177]
 [ 0.61715852  0.87434252 -0.06931573]] nm


In [11]:
# NBVAL_IGNORE_OUTPUT
# covers mdtraj warning
psi = md.compute_psi(traj.to_mdtraj())

/home/sander/miniconda3/envs/python3/lib/python3.9/site-packages/mdtraj/utils/validation.py:115: TypeCastPerformanceWarning: Casting unitcell_vectors dtype=float64 to <class 'numpy.float32'> 
  warnings.warn("Casting %s dtype=%s to %s " % (name, val.dtype, dtype),


In [12]:
# NBVAL_IGNORE_OUTPUT
psi[1][3:8]

array([[ 3.1061454],
       [-3.0958629],
       [ 3.1367555],
       [ 3.1283271],
       [ 3.0931787]], dtype=float32)

In [13]:
st = paths.Storage('engine_store_test.nc', mode='w')

In [14]:
st.engines.save(engine);
st.tag['template'] = template

The engine is stored by using the XML serialization of OpenMM and put this into the storage.

In [15]:
# NBVAL_SKIP
st.variables['engines_json'][0][0:256] + '...'

'{"_cls":"OpenMMEngine","_dict":{"system_xml":"<?xml version=\\"1.0\\" ?>\\n<System openmmVersion=\\"7.7\\" type=\\"System\\" version=\\"1\\">\\n\\t<PeriodicBoxVectors>\\n\\t\\t<A x=\\"2\\" y=\\"0\\" z=\\"0\\"\\/>\\n\\t\\t<B x=\\"0\\" y=\\"2\\" z=\\"0\\"\\/>\\n\\t\\t<C x=\\"0\\" y=\\"0\\" z=\\"2...'

In [16]:
st.save(traj);

In [17]:
st.close()

### try to load the stored engine

In [18]:
st = paths.AnalysisStorage('engine_store_test.nc')

In [19]:
engine = st.engines[0]

Run another 10 steps to test

In [20]:
template = st.tag['template']

In [21]:
traj = engine.generate(template, [paths.LengthEnsemble(10).can_append])

In [22]:
print(traj)

Trajectory[10]


In [23]:
st.close()

And save the trajectory for future use (other tests)

In [24]:
st = paths.Storage('engine_store_test.nc', "a")
st.save(traj)
st.close()